In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from rapidfuzz import process, fuzz

In [14]:
# OUTPUT: Processed_Data.csv

In [15]:
# Reading the cleaned data and product data
df_files = pd.read_csv("Clean_Data.csv")
df_products = pd.read_csv("../Format & Logistics Data/item_no.csv", encoding='latin1')

In [16]:
# --------------------
# Normalize for comparison
# --------------------
df_products['item_no'] = df_products['item_no'].astype(str).str.lower()
df_products['description'] = df_products['description'].astype(str).str.strip()
df_files['file_name'] = df_files['file_name'].astype(str).str.lower()

# --------------------
# Add empty columns for result
# --------------------
df_files['matched_item_no'] = None
df_files['matched_description'] = None
df_files['match_flag'] = 0
df_files['fuzzy_score'] = 0  # Optional: track how good the match is

# --------------------
# Prepare product item list
# --------------------
product_list = df_products[['item_no', 'description']].values.tolist()

# --------------------
# Fuzzy match each unmatched file name with item_no
# --------------------


for idx, file_row in df_files[df_files['match_flag'] == 0].iterrows():
    file_name = file_row['file_name']
    
    # Get best fuzzy match for this file name
    best_match = process.extractOne(
        file_name,
        [item[0] for item in product_list],
        scorer=fuzz.partial_ratio
    )
    
    if best_match:
        best_score = best_match[1]
        if best_score > 0:
            best_item_no = best_match[0]
            best_desc = df_products.loc[df_products['item_no'] == best_item_no, 'description'].values[0]

            df_files.at[idx, 'matched_item_no'] = best_item_no
            df_files.at[idx, 'matched_description'] = best_desc
            df_files.at[idx, 'match_flag'] = 1
            df_files.at[idx, 'fuzzy_partial_ratio'] = best_score



In [17]:
# Rename columns for clarity

columns_to_rename = {
    'parent_folder': 'prefix_name',
    'matched_item_no': 'item_no',
    'matched_description': 'description',
    'match_flag': 'item_no_match_flag'
}

df_files.rename(columns=columns_to_rename, inplace=True)
print(df_files.columns.tolist())


df_files = df_files[['item_no', 'file_name', 'description', 'prefix_name', 'pdf','dwg','sldprt', 'sldasm', 'ipt', 'iam','idw', 'file_path','main_folder', 'item_no_match_flag','fuzzy_partial_ratio']]

df_files


['prefix_name', 'file_name', 'file_path', 'pdf', 'iam', 'ipt', 'dwg', 'rfa', 'sldprt', 'sldasm', 'step', 'idw', 'main_folder', 'item_no', 'description', 'item_no_match_flag', 'fuzzy_score', 'fuzzy_partial_ratio']


,item_no,file_name,description,prefix_name,pdf,dwg,sldprt,sldasm,ipt,iam,idw,file_path,main_folder,item_no_match_flag,fuzzy_partial_ratio
0,plywood,4x9 plywood sheet,PLYWOOD,- 3D PARTS,0,1,0,0,0,0,0,H:\CACLA - Caledon ON\Product and Technical\Fa...,Fabrication,1,100.000000
1,plywood,4x9 plywood sheet 2,PLYWOOD,- 3D PARTS,0,1,0,0,0,0,0,H:\CACLA - Caledon ON\Product and Technical\Fa...,Fabrication,1,100.000000
2,sts,accesory parts,SYS TRUSS SADDLE,- 3D PARTS,0,1,0,0,0,0,1,H:\CACLA - Caledon ON\Product and Technical\Fa...,Fabrication,1,80.000000
3,ll,bro filler ply,LIFTING LUG,- 3D PARTS,0,0,0,0,1,0,0,H:\CACLA - Caledon ON\Product and Technical\Fa...,Fabrication,1,100.000000
4,ll,bro wall,LIFTING LUG,- 3D PARTS,0,0,0,0,1,0,0,H:\CACLA - Caledon ON\Product and Technical\Fa...,Fabrication,1,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76359,racw,corpo_paranco,"CLAMP R/A WEDGE 2"" X 2""",winch--1,0,0,1,0,0,0,0,H:\CACLA - Caledon ON\Product and Technical\Pr...,Prodtech,1,75.000000
76360,ll,gancio_per_paranco_alla catena,LIFTING LUG,winch--1,0,0,1,0,0,0,0,H:\CACLA - Caledon ON\Product and Technical\Pr...,Prodtech,1,100.000000
76361,rcpa,gancio_per_paranco_superiore,RESERVOIR CLAMP PLATE (ALUMA),winch--1,0,0,1,0,0,0,0,H:\CACLA - Caledon ON\Product and Technical\Pr...,Prodtech,1,75.000000
76362,mka2,magliacatena,FEMALE CORNER PLATE,winch--1,0,0,1,0,0,0,0,H:\CACLA - Caledon ON\Product and Technical\Pr...,Prodtech,1,66.666667


In [18]:
df_files.to_csv("Processed_Data.csv", index=False)